<a href="https://colab.research.google.com/github/AtogmatomaProgramming/Android-Expert/blob/main/manage_frames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#=== Importar librerías ===
from google.colab import drive
import os
import cv2

In [ ]:
#=== Montar drive ===
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#=== Rutas carpetas ===
base_video_dir = "/content/drive/My Drive/alpha_training/videos"
output_frame_dir = "/content/drive/My Drive/alpha_training/frames"

In [ ]:
#=== Extracción final de frames ===

# Número de frames que quieres extraer por video
total_frames_desired = 500

# Función para crear una carpeta si no existe
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

# Recorrer las carpetas de especies en la carpeta de videos
for species in os.listdir(base_video_dir):
    species_path = os.path.join(base_video_dir, species)
    if os.path.isdir(species_path):  # Confirmar que es un directorio
        # Crear una carpeta de salida para la especie en 'frames'
        species_output_dir = os.path.join(output_frame_dir, species)
        create_folder_if_not_exists(species_output_dir)

        # Recorrer los videos en la carpeta de cada especie
        for idx, video_file in enumerate(os.listdir(species_path)):
            video_path = os.path.join(species_path, video_file)
            video_code = f'v{idx + 1}'  # Identificar el video (v1, v2, etc.)

            # Verificar si ya existen frames del video en la carpeta de la especie
            existing_frames = [f for f in os.listdir(species_output_dir) if f.startswith(video_code)]
            if existing_frames:
                print(f"Frames ya extraídos para {video_file}, ignorando...")
                continue

            # Procesar el video y calcular el intervalo para 500 frames
            cap = cv2.VideoCapture(video_path)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            interval = max(1, frame_count // total_frames_desired)  # Calcula el intervalo dinámico

            # Extraer frames en intervalos calculados
            current_frame = 0
            saved_frames = 0
            while cap.isOpened() and saved_frames < total_frames_desired:
                ret, frame = cap.read()
                if not ret:
                    break
                # Guardar frames en intervalos específicos
                if current_frame % interval == 0:
                    frame_name = os.path.join(species_output_dir, f'{video_code}_frame{current_frame}.jpg')
                    cv2.imwrite(frame_name, frame)
                    saved_frames += 1
                current_frame += 1

            cap.release()
            print(f"{saved_frames} frames extraídos de {video_file} y guardados en {species_output_dir}")

print("Extracción de frames completada.")